Test to what extent is filter information ( hence wavelength dependence) passed from `ts_phosim` to `ts_wep` - does `Algorithm.py` know about the SED of the fitted soures that created the donuts?  

# `ts_phosim`

We know that when the AOS simulation is run with `imgCloseLoop` the argument [--filterType](https://github.com/lsst-ts/ts_phosim/blob/0858fc632e63686712189eb0ed928d3f84bd5cd6/python/lsst/ts/phosim/CloseLoopTask.py#L1529) if not otherwise specified is set by default to "", which stands for "reference wavelength". Thus by default `FilterType.REF` is returned by [_getFilterType](https://github.com/lsst-ts/ts_phosim/blob/0858fc632e63686712189eb0ed928d3f84bd5cd6/python/lsst/ts/phosim/CloseLoopTask.py#L430) method. 


Whether we `runOpd` only, or the full `runImg` simulation, this information is passed to [configPhosimCmpt](https://github.com/lsst-ts/ts_phosim/blob/0858fc632e63686712189eb0ed928d3f84bd5cd6/python/lsst/ts/phosim/CloseLoopTask.py#L1269), and in turn to [self.phosimCmpt.setSurveyParam](https://github.com/lsst-ts/ts_phosim/blob/0858fc632e63686712189eb0ed928d3f84bd5cd6/python/lsst/ts/phosim/CloseLoopTask.py#L223)

which [PhosimCmpt](https://github.com/lsst-ts/ts_phosim/blob/0858fc632e63686712189eb0ed928d3f84bd5cd6/python/lsst/ts/phosim/PhosimCmpt.py#L294) shares with  [TeleFacade](https://github.com/lsst-ts/ts_phosim/blob/0858fc632e63686712189eb0ed928d3f84bd5cd6/python/lsst/ts/phosim/telescope/TeleFacade.py#L183)

When [writeStarInstFile](https://github.com/lsst-ts/ts_phosim/blob/0858fc632e63686712189eb0ed928d3f84bd5cd6/python/lsst/ts/phosim/telescope/TeleFacade.py#L559) is created,  [_getFilterIdInPhoSim](https://github.com/lsst-ts/ts_phosim/blob/0858fc632e63686712189eb0ed928d3f84bd5cd6/python/lsst/ts/phosim/telescope/TeleFacade.py#L615) uses [mapFilterRefToG](Utility.https://github.com/lsst-ts/ts_wep/blob/a245211b7f3bd0be1e126b4066c45bfbaf6a7e47/python/lsst/ts/wep/Utility.py#L321), which maps 

`FilterType.REF` --> `FilterType.G`


And [phosimCommu.getFilterId](https://github.com/lsst-ts/ts_phosim/blob/0858fc632e63686712189eb0ed928d3f84bd5cd6/python/lsst/ts/phosim/telescope/PhosimCommu.py#L68) is used to translate that `FilterType` to a phosim filter name . For `REF`, or `G`, the `filterId = 1`.


This is stored in the `opd` [instance catalog](https://github.com/lsst-ts/ts_phosim/blob/0858fc632e63686712189eb0ed928d3f84bd5cd6/python/lsst/ts/phosim/telescope/PhosimCommu.py#L465) as `Opsim_filter 1` .


If `filterType == FilterType.REF` an appropriate SED file is written [writeSedFile](https://github.com/lsst-ts/ts_phosim/blob/0858fc632e63686712189eb0ed928d3f84bd5cd6/python/lsst/ts/phosim/telescope/PhosimCommu.py#L557) , which [reads the wavelength to use](https://github.com/lsst-ts/ts_phosim/blob/0858fc632e63686712189eb0ed928d3f84bd5cd6/python/lsst/ts/phosim/telescope/TeleFacade.py#L170) from [teleSetting.yaml](https://github.com/lsst-ts/ts_phosim/blob/0858fc632e63686712189eb0ed928d3f84bd5cd6/policy/teleSetting.yaml#L43). By default, that wavelength is `500 nm` .

Thus the `sedFileName = "sed_500.txt"`, and is stored in  `phosimDir/data/sky/sedFileName`. The content of such file is just one line - all intensity is set to a single wavelength:


    500   1.0 



Then `phosimCommu` when it adds stars to the instance catalog, it uses [phosimCommu.generateStar](https://github.com/lsst-ts/ts_phosim/blob/0858fc632e63686712189eb0ed928d3f84bd5cd6/python/lsst/ts/phosim/telescope/PhosimCommu.py#L319)  which adds 

    content = "object %2d\t%9.6f\t%9.6f %9.6f ../sky/%s " % (
            starId,
            ra,
            dec,
            magNorm,
            sedName,
        )
    content += "%.1f %.1f %.1f %.1f %.1f %.1f %s %.1f none none \n" % (
            redshift,
            gamma1,
            gamma2,
            kappa,
            deltaRa,
            deltaDec,
            sourceType,
            starId,
        )
        
which looks like eg. 

    object  0	 0.066372	 0.098209 15.000000 ../sky/sed_500.txt 0.0 0.0 0.0 0.0 0.0 0.0 star 0.0 none none 
        

that's how each star could (in principle) have a different (and quite complex) SED in phoSim. 


Pass correct filter type in `ts_phosim`. First, make a small  function to map filter name to g if it's a reference filter, and leave unchanged  for any other standard filter name:

In [11]:
def mapFilterRefToG(filterTypeName):
    if filterTypeName in ['ref','']: 
        return 'g'
    else:
        return filterTypeName

In [14]:
for filterTypeName in ['ref','','u','g','r','i','z','y']:
    print(filterTypeName,'-->', mapFilterRefToG(filterTypeName))

ref --> g
 --> g
u --> u
g --> g
r --> r
i --> i
z --> z
y --> y


Also, write correct magnitude limits so that source selector uses different values depending on filter chosen:

In [24]:

from lsst.ts.wep.ParamReader import ParamReader
from lsst.ts.wep.Utility import getConfigDir


#-> get magnitude limits from ts_wep/policy/task/magLimitStar.yaml

configDir = getConfigDir()  # ts_wep config dir - check that it works 
settingFilePath = os.path.join(configDir, "task/magLimitStar.yaml")
magLimitSettingFile = ParamReader(filePath=settingFilePath)
magLimits = magLimitSettingFile.getSetting(f"filter{filterTypeName.upper()}")

This update was submitted as https://github.com/lsst-ts/ts_phosim/pull/79 

# `ts_wep`


In `ts_wep`, the information about the filter is passed in the configuration files passed to the wavefront estimation pipeline.

It is written in [imgCloseLoop.writeWepConfiguration](https://github.com/lsst-ts/ts_phosim/blob/0858fc632e63686712189eb0ed928d3f84bd5cd6/python/lsst/ts/phosim/CloseLoopTask.py#L1148).

The relevant content is 

    generateDonutCatalogWcsTask:
    class: lsst.ts.wep.task.GenerateDonutCatalogWcsTask.GenerateDonutCatalogWcsTask
    # Here we specify the configurations for pointing that we added into the class
    # GenerateDonutCatalogWcsTaskConfig.
    config:
      filterName: 'g'
      referenceSelector.doMagLimit: True
      referenceSelector.magLimit.maximum: 15.90
      referenceSelector.magLimit.minimum: 8.74
      referenceSelector.magLimit.fluxField: 'g_flux'
      doDonutSelection: True
      donutSelector.fluxField: 'g_flux'
      
      
Thus by default `g` filter is hard-coded (**NOTE** - that should not be the case, given that `imgCloseLoop` accepts  `filterType` to be one of `ugrizy` or `REF (=g)`, we should allow this to also depend on the input `filterType`. Unless it doesn't matter...


Filter information is no part of `wfEsti`  [config](https://github.com/lsst-ts/ts_wep/blob/a245211b7f3bd0be1e126b4066c45bfbaf6a7e47/python/lsst/ts/wep/WfEstimator.py#L129) . Thus it is also not part of [Algorithm](https://github.com/lsst-ts/ts_wep/blob/develop/python/lsst/ts/wep/cwfs/Algorithm.py#L647),  and not of [Instrument](https://github.com/lsst-ts/ts_wep/blob/develop/python/lsst/ts/wep/cwfs/Instrument.py).




# `ts_ofc` 


The `FilterType` information is passed to `ts_ofc` in [CloseLoopTask](https://github.com/lsst-ts/ts_phosim/blob/df5bae9f63788ec4a3d66a4d35386c47fdc206ab/python/lsst/ts/phosim/CloseLoopTask.py#L663). [OFCcalc.calculate_correctiobs](https://github.com/lsst-ts/ts_ofc/blob/062a66656a787742f4acd3f9da9b7bbf04587722/python/lsst/ts/ofc/ofc.py#L155)  uses the filter information to get the optical state.
